libraries used

In [22]:
import csv
import PyPDF2 as pypdf
import json
import os
from pdf2image import convert_from_path
import checkboxes


import matplotlib.pyplot as plt

'detect' getting the data for the solutions

In [29]:
#set up the csv reader to get the id of each student
file = open("en/students.csv")
reader = csv.reader(file)
header = next(reader)
students = []

for row in reader:
    #open the pdf with the solutions and process the checkboxes
    students.append(row[2])
    sol_file = "en/solution/solution_" + row[2] + ".pdf"
    pdfobject=open(sol_file,'rb')
    var = pypdf.PdfFileReader(pdfobject).getFields()
    print(var)

    #process and store the checkbox information on the json files
    data = {'checkboxes' : []}
    for box in var:
        data['checkboxes'].append(
            {
                'checkbox' : box.split(sep=':')[2] + ' ' +  box.split(sep=':')[3],      # ejemplo de contenido de box: 1:case:basic-adittion:1,1
                'state' : var[box]['/V'],
                'cords' : [ float(var[box]['/Rect'][0]) , float(var[box]['/Rect'][1]) ]
            }
        )
    jf_name = 'json/sol_' + row[2] + '.json'
    os.makedirs(os.path.dirname(jf_name), exist_ok=True)
    with open(jf_name, 'w') as outfile:
        json.dump(data, outfile, indent=4)

{'1:case:basic-adittion:1,1': {'/T': '1:case:basic-adittion:1,1', '/FT': '/Btn', '/V': '/Off', '/Rect': [89.867, 581.13, 102.583, 593.846]}, '1:case:basic-adittion:1,2': {'/T': '1:case:basic-adittion:1,2', '/FT': '/Btn', '/V': '/Yes', '/Rect': [111.549, 581.13, 124.265, 593.846]}, '1:case:basic-adittion:1,3': {'/T': '1:case:basic-adittion:1,3', '/FT': '/Btn', '/V': '/Off', '/Rect': [133.232, 581.13, 145.948, 593.846]}, '1:case:basic-adittion:1,4': {'/T': '1:case:basic-adittion:1,4', '/FT': '/Btn', '/V': '/Off', '/Rect': [154.914, 581.13, 167.631, 593.846]}, '1:case:basic-adittion:1,5': {'/T': '1:case:basic-adittion:1,5', '/FT': '/Btn', '/V': '/Off', '/Rect': [176.597, 581.13, 189.313, 593.846]}, '1:case:basic-adittion:1,6': {'/T': '1:case:basic-adittion:1,6', '/FT': '/Btn', '/V': '/Off', '/Rect': [198.28, 581.13, 210.996, 593.846]}, '1:case:basic-adittion:1,7': {'/T': '1:case:basic-adittion:1,7', '/FT': '/Btn', '/V': '/Off', '/Rect': [219.962, 581.13, 232.678, 593.846]}, '1:case:basic-

IMPORTANTE: la lista de estudiantes esta desordenada
Como la utilizo para identificar las paginas de scanned, q si estan ordenadas, tengo q ordenarla

In [24]:
print(students)
students = [int(i) for i in students]
students.sort()
students = [str(i) for i in students]
print(students)

['12345', '31416', '271828', '11111', '22222', '33333', '44444', '55555', '666666']
['11111', '12345', '22222', '31416', '33333', '44444', '55555', '271828', '666666']


'analyse': getting the data from the scanned exams
primero lo intento con boxdetect pero no funciona, no detecta todas las casillas (t_scan_boxdetect.ipynb)

In [25]:
#export the pdf's pages to png with 250 dots per inch (dpi)
images = convert_from_path('scanned.pdf', 250)

for i, image in enumerate(images):
    #save each page in a png file
    if not os.path.exists('pngs'):
        os.mkdir('pngs')
    im_file = 'pngs/page_' + str(i) + '.png'
    image.save(im_file)

    #check the checkboxes and wether they're marked
    boxes = checkboxes.find_checkboxes(im_file)
    is_checked = checkboxes.check_marked(im_file, boxes)
    #print(json.dumps(is_checked, indent=4))

    #export to json
    #since it doesnt check which student did which page, assume the scanned file is ordered like the student list
    jf_name = 'json/scan_' + students[i] + '.json'
    os.makedirs(os.path.dirname(jf_name), exist_ok=True)
    with open(jf_name, 'w') as outfile:
        json.dump(is_checked, outfile, indent=4)

'match': read the json files and check whether the marked checkboxes are the same

In [26]:
for student in students:
    #open the json files (solution and scanned) for each student
    sol_file = 'json/sol_' + student + '.json'
    scan_file = 'json/scan_' + student + '.json'
    with open(sol_file) as json_file:
        data_sol = json.load(json_file)
    with open(scan_file) as json_file:
        data_scan = json.load(json_file)

    #check if the number of checkboxes is the same
    if data_sol['checkboxes'].__len__() == data_scan['checkboxes'].__len__():
        comparacion = checkboxes.match(data_sol,data_scan)
        jf_name = 'json/match_' + student + '.json'
        os.makedirs(os.path.dirname(jf_name), exist_ok=True)
        with open(jf_name, 'w') as outfile:
            json.dump(comparacion, outfile, indent=4)
    else:
        print('Error: number of checkboxes not equal')